In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import  matplotlib.pyplot as plt 

In [ ]:
# MultiPose Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
# SinglePose Model
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [ ]:
#Anahtar Noktaları Çizmek

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
 #(int(kx), int(ky)) kullanarak frame üzerinde 6 piksel çapında  bir daire çizer

In [ ]:
#Bu edegler aslında hangi koordinatın hangi diğer koordinata bağlandığını temsil eder.
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
# Anahtar noktaları_with_scores içinde döner ve her bir kişi için draw_connections ve draw_keypoints fonksiyonlarını çağırarak anahtar noktaları ve kenarları frame üzerinde çizer
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 4)

  #np.squeeze anahtar dizisini çıkarmayı kolaylaştırın
  #  Her iki anahtar noktasının güven seviyesi güven seviyesi eşiğinden yüksekse, anahtar noktaların koordinatlarını
 #(int(x1), int(y1)) ve (int(x2), int(y2)) kullanarak frame üzerinde mavi renkte bir çizgi çizer. 

# SinglePose 

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
     # Görüntüyü yeniden boyutlandırmak
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Kurulum girişi ve çıkışı
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # görüntünün anahtar noktalarını tahmin etme işlemi gerçekleştirir.
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Anahtar noktaları oluşturma  
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('e'):
        break
        
cap.release()
cv2.destroyAllWindows()

# MultiPose

In [ ]:


people_count = 0

cap = cv2.VideoCapture("dance2.mp4")

while cap.isOpened():
    ret,frame = cap.read()
    
    # Görüntüyü yeniden boyutlandırmak
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640) 
    input_img = tf.cast(img, dtype=tf.int32)
    #WEB KAMERAMIZDAN ALDIĞIMIZ HER BİR FRAME DÖNGÜ İÇİNDE ALIYORUZ
    
    #Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    # Anahtar noktaları oluşturma 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
        # Ekranda insanlar var mı kontrol et.
    confidence_threshold = 0.1
    num_people = 0
    for person in keypoints_with_scores:
        if person[0][2] > confidence_threshold:
            num_people += 1
    if num_people == 0:
        people_count = 0
    elif num_people > people_count:
        people_count = num_people
    cv2.putText(frame, f"People: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    
    cv2.imshow("Movenet Multipose", frame)

    if cv2.waitKey(10) & 0xFF==ord("e"):
        break
cap.release()
cv2.destroyAllWindows()
